<a href="https://colab.research.google.com/github/toshineb/amazon_discounts-webscrping/blob/main/amazon_discounts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [6]:
url = "https://www.amazon.com/primeday?pd_rd_w=e7mp3&content-id=amzn1.sym.ee9a48e8-4341-4c6c-87e2-22817bd974de&pf_rd_p=ee9a48e8-4341-4c6c-87e2-22817bd974de&pf_rd_r=BY25GZF4NTPQK5XAJ2RK&pd_rd_wg=T5G5D&pd_rd_r=4fcd7905-9bd2-4c00-8353-28c61a140cf5&ref_=pd_hp_d_btf_unk_cta"
headers = {"User-Agent": "Mozilla/5.0"}
html_content = requests.get(url, headers=headers).text

In [7]:
# === STEP 2: Parse HTML ===
soup = BeautifulSoup(html_content, "lxml")


In [8]:
# === STEP 3: Extract product details ===
products = []
cards = soup.find_all("li", class_="a-carousel-card")

for card in cards:
    try:
        name_tag = card.select_one(".dcl-product-label span")
        price_tag = card.select_one(".dcl-product-price-new .a-offscreen")
        discount_tag = card.select_one("._badgeLabel_1frb3_1 span")

        name = name_tag.text.strip() if name_tag else "N/A"
        price = price_tag.text.strip().replace("$", "") if price_tag else "N/A"
        discount = discount_tag.text.strip() if discount_tag else "N/A"

        products.append({
            "Product Name": name,
            "Price ($)": price,
            "Discount": discount
        })

    except Exception as e:
        print("Error processing a card:", e)


In [9]:
# === STEP 4: Export to Excel ===
df = pd.DataFrame(products)
df.to_excel("amazon_discounts.xlsx", index=False)
print("Data saved to 'amazon_discounts.xlsx'")

✅ Data saved to 'amazon_discounts.xlsx'


# **Second Data of non Prime SubscribSubscribers**

In [10]:
# Load the local HTML file
with open("amazon_primeday.html", "r", encoding="utf-8") as file:
    html = file.read()

soup = BeautifulSoup(html, "html.parser")

data = []

In [11]:
# Loop through each product
for item in soup.select("li.a-carousel-card"):
    try:
        name = item.select_one(".dcl-truncate span").get_text(strip=True)
    except:
        name = None

    try:
        price = item.select_one(".a-price .a-offscreen").get_text(strip=True)
    except:
        price = None

    try:
        discount = item.select_one("._badgeLabel_1frb3_1 span").get_text(strip=True)
    except:
        discount = None

    # Only append if name or price is found
    if name or price:
        data.append({
            "Product Name": name,
            "Price": price,
            "Discount": discount
        })


In [12]:
# Convert to Excel
df = pd.DataFrame(data)
df.to_excel("primeday_products.xlsx", index=False)

print("Extracted and saved to primeday_products.xlsx")

Extracted and saved to primeday_products.xlsx
